# Prosta analiza tekstów Juliusza Słowackiego

In [12]:
import os, fnmatch
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.feature_extraction.text import CountVectorizer
# zmiana sposobu wyświetlania danych typu float
pd.options.display.float_format = "{:.3f}".format 

#### Utilities

In [13]:
stops = []
limiters = ["\\n", "'"]
with open('stopyPL.txt', encoding='utf-8') as sf:
    for sline in sf:
        sline = repr(sline)
        for char in limiters:
            sline = sline.replace(char,"")
        if not(sline == ""):
            stops.append(sline)
            
unnecessary_chars = ["!",",",".","    ",":",";","?","(",")","—","*","»","…","«","—",'-','„','”','"','°']

#### Wybieranie z katalogu wszystkich książek danego autora

In [14]:
def findAllBooks(author, path):
    result = []
    for root, dirs, files in os.walk(path):
        for name in files:
            if fnmatch.fnmatch(name, author + '_*.txt'):
                result.append(os.path.join(root, name))
    return result

testauthor = 'juliusz-slowacki'
testbooklist = findAllBooks(testauthor, '..\\test_books\\slowacki_books')
print('Parsing check: is the return value a list of valid paths?', os.path.isfile(testbooklist[0]))

testbooklist

Parsing check: is the return value a list of valid paths? True


['..\\test_books\\slowacki_books\\juliusz-slowacki_a-jednak-ja-nie-watpie-bo-sie-pora-zbliza.txt',
 '..\\test_books\\slowacki_books\\juliusz-slowacki_aniol-ognisty-moj-aniol-lewy.txt',
 '..\\test_books\\slowacki_books\\juliusz-slowacki_balladyna.txt',
 '..\\test_books\\slowacki_books\\juliusz-slowacki_do-matki-w-ciemnosciach-postac-mi-stoi-matczyna.txt',
 '..\\test_books\\slowacki_books\\juliusz-slowacki_do-matki.txt',
 '..\\test_books\\slowacki_books\\juliusz-slowacki_do-pani-joanny-bobrowej.txt',
 '..\\test_books\\slowacki_books\\juliusz-slowacki_do-pastereczki-siedzacej-na-druidow-kamieniach-w-pornic-nad-oceanem.txt',
 '..\\test_books\\slowacki_books\\juliusz-slowacki_do-teofila-januszewskiego.txt',
 '..\\test_books\\slowacki_books\\juliusz-slowacki_do-zygmunta.txt',
 '..\\test_books\\slowacki_books\\juliusz-slowacki_duma-o-waclawie-rzewuskim.txt',
 '..\\test_books\\slowacki_books\\juliusz-slowacki_dusza-sie-moja-zamysla-gleboko.txt',
 '..\\test_books\\slowacki_books\\juliusz-slowac

Pokazywanie słownika z progiem

In [15]:
def pokaz_czeste(slownik,prog):
    for klucz,wartosc in slownik.items():
        if wartosc>prog:
            print(klucz,":",wartosc)

Pokazywanie słownika z zakresem

In [16]:
def pokaz_slownik(slownik,zakres):
    dlugosc = len(slownik)
    if zakres[0]<0: zakres[0]=0
    if zakres[1]>dlugosc: zakres[1]=dlugosc
    if zakres[1]>zakres[0]:
        i = 0
        for klucz,wartosc in slownik.items():
            if i>=zakres[0] and i<zakres[1]:
                print(klucz,":",wartosc)
            i = i+1

#### Tworzenie tokenów, usuwanie stop-słów, tworzenie słownika ilości wszystkich tokenów 

Tworzymy słownik dla 50 wczytanych utworów Juliusza Słowackiego i liczymy częstość dla każdego słowa.

In [17]:
tokens_stop = []
for pathToBook in testbooklist:
    with open(pathToBook, 'r', encoding='utf-8') as book:
        for line in book:        
            for char in unnecessary_chars:
                line = line.replace(char, '')
            line = line.rstrip().lower().split()
            
            if line:
                tokens_stop.extend(line)
                
    tokens = []
    for ts in tokens_stop:
        if not ts in stops:
            tokens.append(ts)
            
    dictionary = dict()
    for word in sorted(tokens):
        if word.isalpha():
            dictionary[word] = dictionary.get(word, 0) + 1

Najczęściej występujące słowa

In [18]:
po_czestosci = dict(sorted(dictionary.items(), key=lambda x: x[1], reverse = True))
pokaz_slownik(po_czestosci,[0,20])

balladyna : 273
kordian : 176
kirkor : 113
grabiec : 109
goplana : 104
wdowa : 103
kostryn : 91
pustelnik : 86
mię : 73
niechaj : 72
skierka : 71
boga : 69
ludu : 66
boże : 64
wielki : 63
bóg : 62
potem : 62
serce : 62
cha : 61
oczy : 60


Nadłuższe słowa

In [19]:
po_dlugosci = dict(sorted(dictionary.items(), key=lambda x: len(x[0]), reverse = True ))
pokaz_slownik(po_dlugosci,[0,20])

czterdziestogodzinne : 1
nieprzewidzianych : 1
niesprawiedliwość : 1
szczodrobliwością : 1
zmartwychwstaniem : 1
zmartwychwstałego : 1
śnieżystobłękitne : 1
dwudziestoletnie : 1
niecierpliwością : 1
niedoścignionych : 1
nieskończonością : 1
przedzierżgniono : 1
zmartwychwstania : 4
zmartwychwstanie : 2
światotworzącymi : 1
czarnoksiężnica : 1
czarnoksiężnika : 1
czarodziejskich : 1
konferencyjnego : 2
nadprzyrodzonej : 1


Liczba unikalnych tokenów

In [20]:
len(dictionary)

14207